In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# 기본 웹드라이버로 Chrome 실행
driver = webdriver.Chrome()

# URL 접속
url = 'https://www.lexus.co.kr/service/service-center/'
driver.get(url)

# 데이터 저장을 위한 빈 리스트 생성
service_centers = []

# 각 지역 탭 선택 및 클릭
region_tabs = driver.find_elements(By.CSS_SELECTOR, '.sub-menu-list.m-sub-list .menu li a')
for tab in region_tabs:
    tab.click()  # 탭을 클릭하여 활성화
    time.sleep(2)  # 페이지 갱신 대기

    # 각 탭에서 서비스 센터 목록을 가져옴
    service_centers_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.mod-find-accordion"))
    )
    
    # 각 서비스 센터의 정보를 순차적으로 수집
    for center in service_centers_elements:
        # 센터명
        center_name = None
        if center.find_elements(By.TAG_NAME, "dt"):
            center_name = center.find_element(By.TAG_NAME, "dt").text.strip()
        
        # 정비내용
        service_type = None
        if center.find_elements(By.CSS_SELECTOR, "dd.dealer-service span"):
            service_type = center.find_element(By.CSS_SELECTOR, "dd.dealer-service span").text.strip()
        
        # 주소 
        address = None
        if center.find_elements(By.CSS_SELECTOR, "dd.dealer-addr > span"):
            address = center.find_element(By.CSS_SELECTOR, "dd.dealer-addr > span").text.strip()
        
        # 전화번호
        tel = None
        if center.find_elements(By.CSS_SELECTOR, "dd.dealer-tel span:nth-of-type(2)"):
            tel = center.find_element(By.CSS_SELECTOR, "dd.dealer-tel span:nth-of-type(2)").text.strip()
        
        # FAX
        fax = None
        if center.find_elements(By.CSS_SELECTOR, "dd.dealer-fax span.txt-data"):
            fax = center.find_element(By.CSS_SELECTOR, "dd.dealer-fax span.txt-data").text.strip()
        
        # 각 센터의 정보를 딕셔너리에 저장
        service_centers.append({
            "센터명": center_name,
            "정비내용": service_type,
            "주소": address,
            "전화번호": tel,
            "FAX": fax
        })

# DataFrame 생성
df = pd.DataFrame(service_centers)

df = df[df['센터명'].str.contains("[가-힣]", na=False)]
df

df.reset_index(drop=True, inplace=True)
df['주소'] = df['주소'].str.replace('특별시|광역시|특별자치도|특별자치시', '', regex=True)
df['주소'] = df['주소'].str.replace('서울시', '서울', regex=False)
df['주소'] = df['주소'].str.replace('경기도', '경기', regex=False)
df['주소'] = df['주소'].str.replace('강원도', '강원', regex=False)
df['주소'] = df['주소'].str.replace('충청북도', '충북', regex=False)
df['주소'] = df['주소'].str.replace('충청남도', '충남', regex=False)
df['주소'] = df['주소'].str.replace('전라북도', '전북', regex=False)
df['주소'] = df['주소'].str.replace('전라남도', '전남', regex=False)
df['주소'] = df['주소'].str.replace('경상북도', '경북', regex=False)
df['주소'] = df['주소'].str.replace('경상남도', '경남', regex=False)

# '시도' 열에 주소 첫 두 글자 추가
df['시도'] = df['주소'].str[:2]

# '시군구' 열에 첫 번째와 두 번째 띄어쓰기 사이의 텍스트 추출
df['시군구'] = df['주소'].str.extract(r'^\S+\s(\S+)')[0]

df['시도+시군구'] = df['시도'] + df['시군구']

df.to_excel('렉서스_최종.xlsx')